In [2]:
import io  # 文件数据流
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
# 导入常见网络层, sequential容器, 优化器, 损失函数
from tensorflow.keras import layers, Sequential, optimizers, losses, metrics
import os # 运维模块， 调用系统命令
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 只显示warring和error

In [3]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y


def plot_to_image(figure):
    buf = io.BytesIO()  # 在内存中存储画
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    # 传化为TF 图
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image


def image_grid(images):
    # 返回一个5x5的mnist图像
    figure  = plt.figure(figsize=(10, 10))
    for i in range(25):
        plt.subplot(5, 5, i+1, title='name')
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
    return figure

In [4]:
batchsz = 128
path = r'./mnist.npz'
(x, y), (x_val, y_val) = tf.keras.datasets.mnist.load_data(path)
print('datasets:', x.shape, y.shape, x.min(), x.max())

11493376/11490434 [==============================] - 0s 0us/step
datasets: (60000, 28, 28) (60000,) 0 255


In [5]:
db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(preprocess).shuffle(60000).batch(batchsz).repeat(10)
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_val = ds_val.map(preprocess).batch(batchsz, drop_remainder=True)

In [6]:
network = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)
])

network.build(input_shape=(None, 28*28))
network.summary()
optimizer=optimizers.Adam(lr=0.01)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  32896     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  2080      
_________________________________________________________________
dense_4 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [7]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/' + current_time
summary_writer = tf.summary.create_file_writer(log_dir)  # 创建监控类，监控数据写入到log_dir目录

sample_img = next(iter(db))[0]
sample_img = sample_img[0]  # 第一张图
sample_img = tf.reshape(sample_img, [1, 28, 28, 1])
with summary_writer.as_default():  # 写入环境
    tf.summary.image("Training sample:", sample_img, step=0)

In [8]:
for step, (x, y) in enumerate(db):    # 遍历切分好的数据step:0->599
    with tf.GradientTape() as tape:
        x = tf.reshape(x, (-1, 28*28))
        out = network(x)
        y = tf.one_hot(y, depth=10)
        loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y, out, from_logits=True))

    grads = tape.gradient(loss, network.trainable_variables)
    optimizer.apply_gradients(zip(grads, network.trainable_variables))

    if step % 100 == 0:
        print(step, 'loss:', float(loss))  # 读统计数据
        with summary_writer.as_default():
            tf.summary.scalar('train-loss', float(loss), step=step)  # 将loss写入到train-loss中

    if step % 500 == 0:
        total, total_correct = 0., 0

        for _, (m, n) in enumerate(ds_val):
            m = tf.reshape(m, (-1, 28*28))
            out = network(m)
            pred = tf.argmax(out, axis=1)
            pred = tf.cast(pred, dtype=tf.int32)
            correct = tf.equal(pred, n)
            total_correct += tf.reduce_sum(tf.cast(correct, dtype=tf.int32)).numpy()
            total += m.shape[0]

        print(step, 'Evaluate Acc:', total_correct / total)

        val_images = m[:25]
        val_images = tf.reshape(val_images, [-1, 28, 28, 1])
        with summary_writer.as_default():
            tf.summary.scalar('test-acc', float(total_correct / total), step=step)  # 写入测试准确率
            tf.summary.image("val-onebyone-images:", val_images, max_outputs=25, step=step)  # 可视化测试用图片，25张
            val_images = tf.reshape(val_images, [-1, 28, 28])
            figure = image_grid(val_images)
            tf.summary.image('val-images:', plot_to_image(figure), step=step)

0 loss: 2.326235771179199
0 Evaluate Acc: 0.1346153846153846
100 loss: 0.40445271134376526
200 loss: 0.17955899238586426
300 loss: 0.2395472228527069
400 loss: 0.22995679080486298
500 loss: 0.10855032503604889
500 Evaluate Acc: 0.9526241987179487
600 loss: 0.11299649626016617
700 loss: 0.18857276439666748
800 loss: 0.09502774477005005
900 loss: 0.12557914853096008
1000 loss: 0.10354170203208923
1000 Evaluate Acc: 0.9581330128205128
1100 loss: 0.06277208030223846
1200 loss: 0.15149468183517456
1300 loss: 0.1416153907775879
1400 loss: 0.037692729383707047
1500 loss: 0.09318254888057709
1500 Evaluate Acc: 0.9625400641025641
1600 loss: 0.13104911148548126
1700 loss: 0.0459851510822773
1800 loss: 0.087934710085392
1900 loss: 0.016891684383153915
2000 loss: 0.07304257899522781
2000 Evaluate Acc: 0.9686498397435898
2100 loss: 0.1368153691291809
2200 loss: 0.18466390669345856
2300 loss: 0.10615689307451248
2400 loss: 0.07346498966217041
2500 loss: 0.11300792545080185
2500 Evaluate Acc: 0.96744

In [ ]:
!tensorboard --logdir "./log"